# Load Data từ Bronze Layer sang Silver Layer

Notebook này sẽ đọc dữ liệu từ Bronze layer (MinIO) và xử lý để load vào các bảng Iceberg trong Silver layer với Nessie catalog.

## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
from datetime import datetime
import os
import hashlib  # Để dùng trong batch processing nếu cần

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder
    # .master("spark://spark-master:7077") # để chạy DAG bên Spark Cluster
    .appName("Load_Bronze_To_Silver")
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
print("Spark Session đã được khởi tạo với Nessie catalog!")

Spark Session đã được khởi tạo với Nessie catalog!


## 2. Load Bảng SCHOOL

In [40]:
print("=" * 80)
print("LOAD BẢNG SCHOOL")
print("=" * 80)

# Đọc và merge tất cả các năm
years = [2021, 2022, 2023, 2024, 2025]
base_path = "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_"
df_school = spark.read.option("header", "true").option("inferSchema", "true").csv([f"{base_path}{year}.csv" for year in years]).select("TenTruong", "MaTruong", "TinhThanh").dropDuplicates()

# Transform
df_school_silver = df_school.select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("TenTruong").cast("string").alias("schoolName"),
    col("TinhThanh").cast("string").alias("province"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("schoolId").isNotNull() & col("schoolName").isNotNull())

# Ghi vào Silver
df_school_silver.writeTo("nessie.silver_tables.school").using("iceberg").createOrReplace()
print(f"Đã ghi {df_school_silver.count()} dòng vào school")

# Verify
spark.table("nessie.silver_tables.school").show(5, truncate=False)

LOAD BẢNG SCHOOL
Đã ghi 265 dòng vào school
+--------+--------------------------------------+-----------+--------------------------+--------------------------+
|schoolId|schoolName                            |province   |created_at                |updated_at                |
+--------+--------------------------------------+-----------+--------------------------+--------------------------+
|DHF     |Đại học Ngoại Ngữ - Đại học Huế       |Huế        |2025-11-28 04:36:13.764956|2025-11-28 04:36:13.764956|
|DVB     |Đại học Việt Bắc                      |Thái Nguyên|2025-11-28 04:36:13.764956|2025-11-28 04:36:13.764956|
|DCQ     |Đại học Công Nghệ và Quản Lý Hữu Nghị |Hà Nội     |2025-11-28 04:36:13.764956|2025-11-28 04:36:13.764956|
|NTT     |Đại học Nguyễn Tất Thành              |TP HCM     |2025-11-28 04:36:13.764956|2025-11-28 04:36:13.764956|
|KGH     |Trường Sĩ Quan Không Quân - Hệ Đại học|Khánh Hòa  |2025-11-28 04:36:13.764956|2025-11-28 04:36:13.764956|
+--------+------------------

## 3. Load Bảng MAJOR

In [41]:
from pyspark.sql.functions import col, lower, trim, regexp_replace, current_timestamp

df_major = spark.read.option("header", "true") \
    .option("inferSchema", "false") \
    .option("encoding", "UTF-8") \
    .csv("s3a://bronze/structured_data/danh sách các ngành đại học/Danh_sách_các_ngành.csv")

df_major_clean = df_major.select(
    regexp_replace(trim(col(df_major.columns[0])).cast("string"), r"\.0$", "").alias("majorId"),
    trim(col(df_major.columns[1])).cast("string").alias("majorName")
).filter(
    (col("majorId").isNotNull()) &
    (col("majorName").isNotNull()) &
    (col("majorId") != "") &
    (col("majorName") != "") &
    (lower(col("majorId")) != "nan")
)

# Chuẩn hoá để dedupe theo lowercase
df_major_silver = df_major_clean \
    .withColumn("majorId_lower", lower(col("majorId"))) \
    .dropDuplicates(["majorId_lower"]) \
    .select(
        col("majorId"),
        col("majorName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )

df_major_silver.writeTo("nessie.silver_tables.major").using("iceberg").createOrReplace()

print(f"Đã ghi {df_major_silver.count()} dòng vào major")
spark.table("nessie.silver_tables.major").show(5, truncate=False)


Đã ghi 3085 dòng vào major
+-------+------------------------------------------------------------+--------------------------+--------------------------+
|majorId|majorName                                                   |created_at                |updated_at                |
+-------+------------------------------------------------------------+--------------------------+--------------------------+
|106    |Khoa học Máy tính                                           |2025-11-28 04:36:14.993575|2025-11-28 04:36:14.993575|
|107    |Kỹ thuật Máy tính                                           |2025-11-28 04:36:14.993575|2025-11-28 04:36:14.993575|
|108    |Điện - Điện tử - Viễn Thông - Tự động hoá - Thiết kế vi mạch|2025-11-28 04:36:14.993575|2025-11-28 04:36:14.993575|
|109    |Kỹ Thuật Cơ khí                                             |2025-11-28 04:36:14.993575|2025-11-28 04:36:14.993575|
|110    |Kỹ Thuật Cơ Điện tử                                         |2025-11-28 04:36:14.993575|2

## 4. Load Bảng SUBJECT_GROUP và SUBJECT

In [42]:
print("=" * 80)
print("LOAD BẢNG SUBJECT_GROUP và SUBJECT")
print("=" * 80)

# Đọc file tohop_mon_fixed.csv
df_tohop = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/tohop_mon_fixed.csv")

# --- SUBJECT_GROUP ---
df_subject_group_silver = df_tohop.select(
    col(df_tohop.columns[0]).cast("int").alias("subjectGroupId"),
    col(df_tohop.columns[1]).cast("string").alias("subjectGroupName"),
    col(df_tohop.columns[2]).cast("string").alias("subjectCombination"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("subjectGroupId").isNotNull() & col("subjectGroupName").isNotNull() & col("subjectCombination").isNotNull()).dropDuplicates(["subjectGroupName", "subjectCombination"])
df_subject_group_silver.writeTo("nessie.silver_tables.subject_group").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_group_silver.count()} dòng vào subject_group")

# --- SUBJECT ---
df_subject = (
    df_tohop.select(explode(split(col(df_tohop.columns[2]), "-")).alias("subjectName"))
            .withColumn("subjectName", trim(col("subjectName")))
            .filter(col("subjectName").isNotNull() & (col("subjectName") != ""))
            .withColumn("subjectName_lower", lower(col("subjectName")))
            # loại bỏ trùng theo chữ thường
            .dropDuplicates(["subjectName_lower"])
)

window_spec = Window.orderBy("subjectName_lower")
df_subject_silver = df_subject.withColumn("subjectId", row_number().over(window_spec)).select(
    col("subjectId").cast("int"),
    col("subjectName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_subject_silver.writeTo("nessie.silver_tables.subject").using("iceberg").createOrReplace()
print(f"Đã ghi {df_subject_silver.count()} dòng vào subject")

# Verify
spark.table("nessie.silver_tables.subject_group").orderBy("subjectGroupId").show(5, truncate=False)
spark.table("nessie.silver_tables.subject").show(5, truncate=False)

LOAD BẢNG SUBJECT_GROUP và SUBJECT
Đã ghi 232 dòng vào subject_group
Đã ghi 51 dòng vào subject
+--------------+----------------+------------------+--------------------------+--------------------------+
|subjectGroupId|subjectGroupName|subjectCombination|created_at                |updated_at                |
+--------------+----------------+------------------+--------------------------+--------------------------+
|1             |A00             |Toán-Lí-Hóa       |2025-11-28 04:36:16.067063|2025-11-28 04:36:16.067063|
|2             |A01             |Toán-Lí-Ngoại ngữ |2025-11-28 04:36:16.067063|2025-11-28 04:36:16.067063|
|3             |A02             |Toán-Lí-Sinh      |2025-11-28 04:36:16.067063|2025-11-28 04:36:16.067063|
|4             |A03             |Toán-Lí-Sử        |2025-11-28 04:36:16.067063|2025-11-28 04:36:16.067063|
|5             |A04             |Toán-Lí-Địa       |2025-11-28 04:36:16.067063|2025-11-28 04:36:16.067063|
+--------------+----------------+---------------

## 5. Load Bảng SELECTION_METHOD

In [43]:
print("=" * 80)
print("LOAD BẢNG SELECTION_METHOD")
print("=" * 80)

# Đọc từ file benchmark để lấy các phương thức xét tuyển
df_benchmark = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")

# Lấy PhuongThuc và loại bỏ "năm ..."
df_selection = df_benchmark.select(trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")).alias("selectionMethodName")).filter(col("selectionMethodName").isNotNull() & (col("selectionMethodName") != "")).distinct()

window_spec = Window.orderBy("selectionMethodName")
df_selection_method_silver = df_selection.withColumn("selectionMethodId", row_number().over(window_spec)).select(
    col("selectionMethodId").cast("int"),
    col("selectionMethodName").cast("string"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
)
df_selection_method_silver.writeTo("nessie.silver_tables.selection_method").using("iceberg").createOrReplace()
print(f"Đã ghi {df_selection_method_silver.count()} dòng vào selection_method")

# Verify
spark.table("nessie.silver_tables.selection_method").show(10, truncate=False)

LOAD BẢNG SELECTION_METHOD
Đã ghi 10 dòng vào selection_method
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|selectionMethodId|selectionMethodName                                   |created_at                |updated_at                |
+-----------------+------------------------------------------------------+--------------------------+--------------------------+
|1                |Điểm chuẩn theo phương thức Điểm học bạ               |2025-11-28 04:36:18.121363|2025-11-28 04:36:18.121363|
|2                |Điểm chuẩn theo phương thức Điểm thi THPT             |2025-11-28 04:36:18.121363|2025-11-28 04:36:18.121363|
|3                |Điểm chuẩn theo phương thức Điểm xét tuyển kết hợp    |2025-11-28 04:36:18.121363|2025-11-28 04:36:18.121363|
|4                |Điểm chuẩn theo phương thức Điểm xét tốt nghiệp THPT  |2025-11-28 04:36:18.121363|2025-11-28 04:36:18.121363|
|5                |Điểm chuẩn theo

## 6. Load Bảng GradingScale

In [44]:
print("=" * 80)
print("LOAD BẢNG GRADING_SCALE TỪ PHANLOAITHANGDIEM")
print("=" * 80)

# 1. Đọc dữ liệu gốc từ file CSV (giống benchmark)
df_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "true")
        .option("encoding", "UTF-8")
        .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
)

# 2. Lấy unique PhanLoaiThangDiem
df_grading_raw = (
    df_raw
        .select(trim(col("PhanLoaiThangDiem")).alias("description"))
        .filter(col("description").isNotNull() & (col("description") != ""))
        .dropDuplicates(["description"])
)

# 3. Tách giá trị số trong description làm "value" (nếu có, vd: "thang 40" -> 40)
df_grading = (
    df_grading_raw
        .withColumn(
            "value",
            regexp_extract(col("description"), r"(\d+(?:\.\d+)?)", 1).cast("float")
        )
        .withColumn("gradingScaleId", monotonically_increasing_id().cast("int"))
        .withColumn("created_at", current_timestamp())
        .withColumn("updated_at", current_timestamp())
        .select(
            "gradingScaleId",
            "value",
            "description",
            "created_at",
            "updated_at"
        )
)

# 4. Ghi vào bảng Iceberg grading_scale đã tạo trước đó
df_grading.writeTo("nessie.silver_tables.grading_scale") \
          .using("iceberg") \
          .createOrReplace()

print(f"Đã ghi {df_grading.count()} dòng vào grading_scale")

# 5. Verify
spark.table("nessie.silver_tables.grading_scale").show(truncate=False)


LOAD BẢNG GRADING_SCALE TỪ PHANLOAITHANGDIEM
Đã ghi 10 dòng vào grading_scale
+--------------+------+---------------+--------------------------+--------------------------+
|gradingScaleId|value |description    |created_at                |updated_at                |
+--------------+------+---------------+--------------------------+--------------------------+
|0             |30.0  |Thang điểm 30  |2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|1             |1200.0|Thang điểm 1200|2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|2             |40.0  |Thang điểm 40  |2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|3             |50.0  |Thang điểm 50  |2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|4             |150.0 |Thang điểm 150 |2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|5             |10.0  |Thang điểm 10  |2025-11-28 04:36:20.225053|2025-11-28 04:36:20.225053|
|6             |100.0 |Thang điểm 100 |2025-11-28 04:36:20.225053|2025-11-28

## 6. Load Bảng BENCHMARK

In [45]:
from pyspark.sql.functions import (
    col, trim, regexp_replace, current_timestamp, row_number
)
from pyspark.sql.window import Window

print("=" * 80)
print("LOAD BẢNG BENCHMARK")
print("=" * 80)

# Đọc dữ liệu
df_benchmark = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .option("encoding", "UTF-8")
    .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
)

# Xử lý PhuongThuc
df_benchmark = df_benchmark.withColumn(
    "PhuongThuc_cleaned",
    trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", ""))
)

# Join lookup tables
df_selection_lookup     = spark.table("nessie.silver_tables.selection_method")
df_subject_group_lookup = spark.table("nessie.silver_tables.subject_group")
df_grading_scale_lookup = spark.table("nessie.silver_tables.grading_scale")

# Bước 1: chuẩn hóa dữ liệu, chưa tạo ID
df_benchmark_base = (
    df_benchmark
    .join(
        df_selection_lookup,
        df_benchmark["PhuongThuc_cleaned"] == df_selection_lookup["selectionMethodName"],
        "left"
    )
    .join(
        df_subject_group_lookup,
        df_benchmark["KhoiThi"] == df_subject_group_lookup["subjectGroupName"],
        "left"
    )
    .join(
        df_grading_scale_lookup,
        trim(df_benchmark["PhanLoaiThangDiem"]) == df_grading_scale_lookup["description"],
        "left"
    )
    .select(
        col("MaTruong").cast("string").alias("schoolId"),
        col("MaNganh").cast("string").alias("majorId"),
        col("subjectGroupId").cast("int"),
        col("selectionMethodId").cast("int"),
        col("gradingScaleId").cast("int"),
        col("Nam").cast("int").alias("year"),
        col("DiemChuan").cast("double").alias("score"),
        # current_timestamp().alias("created_at"),
        # current_timestamp().alias("updated_at")
    )
    .filter(
        col("schoolId").isNotNull() &
        col("majorId").isNotNull() &
        col("gradingScaleId").isNotNull() &
        col("year").isNotNull() &
        col("score").isNotNull() &
        col("selectionMethodId").isNotNull() 
        #col("subjectGroupId").isNotNull()
    )
    .dropDuplicates([
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "year",
        "gradingScaleId",
        "score"
    ])
)
# ✅ Bước 1.1: GROUP BY và LẤY TRUNG BÌNH SCORE
# Với mỗi (schoolId, majorId, subjectGroupId, selectionMethodId, gradingScaleId, year)
# chỉ còn 1 dòng, score = AVG(score)
df_benchmark_grouped = (
    df_benchmark_base
    .groupBy(
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "gradingScaleId",
        "year"
    )
    .agg(
        round(avg("score"), 2).alias("score")
    )
    .withColumn("created_at", current_timestamp())
    .withColumn("updated_at", current_timestamp())
)

# Bước 2: tạo ID tăng dần đều
window_spec = Window.orderBy(
    "schoolId",
    "majorId",
    "subjectGroupId",
    "selectionMethodId",
    "gradingScaleId",
    "year"
)

df_benchmark_silver = (
    df_benchmark_grouped
    .withColumn("benchmarkId", row_number().over(window_spec).cast("int"))
    .select(
        "benchmarkId",
        "schoolId",
        "majorId",
        "subjectGroupId",
        "selectionMethodId",
        "gradingScaleId",
        "year",
        "score",
        "created_at",
        "updated_at"
    )
)

# Ghi xuống bảng Silver
df_benchmark_silver.writeTo("nessie.silver_tables.benchmark").using("iceberg").createOrReplace()

print(f"Đã ghi {df_benchmark_silver.count()} dòng vào benchmark")

# Verify
spark.table("nessie.silver_tables.benchmark").show(5, truncate=False)
spark.table("nessie.silver_tables.benchmark").groupBy("year").count().orderBy("year").show()


LOAD BẢNG BENCHMARK
Đã ghi 163399 dòng vào benchmark
+-----------+--------+-------+--------------+-----------------+--------------+----+-----+--------------------------+--------------------------+
|benchmarkId|schoolId|majorId|subjectGroupId|selectionMethodId|gradingScaleId|year|score|created_at                |updated_at                |
+-----------+--------+-------+--------------+-----------------+--------------+----+-----+--------------------------+--------------------------+
|1          |ANH     |7480201|1             |2                |0             |2025|21.98|2025-11-28 04:37:01.871063|2025-11-28 04:37:01.871063|
|2          |ANH     |7480201|2             |2                |0             |2025|21.98|2025-11-28 04:37:01.871063|2025-11-28 04:37:01.871063|
|3          |ANH     |7480201|198           |2                |0             |2025|21.98|2025-11-28 04:37:01.871063|2025-11-28 04:37:01.871063|
|4          |ANH     |7480201|199           |2                |0             |2025|

## 7. Load Bảng REGION

In [35]:
print("=" * 80)
print("LOAD BẢNG REGION")
print("=" * 80)

df_region = spark.read.option("header", "true").option("inferSchema", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/region.csv")
df_region_silver = df_region.select(
    lpad(col(df_region.columns[0]).cast("string"), 2, "0").alias("regionId"),  # Format thành 2 chữ số: "1" -> "01"
    col(df_region.columns[1]).cast("string").alias("regionName"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("regionId").isNotNull() & col("regionName").isNotNull()).dropDuplicates(["regionId"])

df_region_silver.writeTo("nessie.silver_tables.region").using("iceberg").createOrReplace()
print(f"Đã ghi {df_region_silver.count()} dòng vào region")

# Verify
spark.table("nessie.silver_tables.region").show(10, truncate=False)

LOAD BẢNG REGION
Đã ghi 64 dòng vào region
+--------+-----------------------+--------------------------+--------------------------+
|regionId|regionName             |created_at                |updated_at                |
+--------+-----------------------+--------------------------+--------------------------+
|01      |Sở GDĐT Hà Nội         |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|02      |Sở GDĐT TP. Hồ Chí Minh|2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|03      |Sở GDĐT Hải Phòng      |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|04      |Sở GDĐT Đà Nẵng        |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|05      |Sở GDĐT Hà Giang       |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|06      |Sở GDĐT Cao Bằng       |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|07      |Sở GDĐT Lai Châu       |2025-11-28 04:35:05.581915|2025-11-28 04:35:05.581915|
|08      |Sở GDĐT Lào Cai        |2025-11-28 04:35:05.581915|2025-1

## 8. Load Bảng STUDENT_SCORES

In [36]:
print("=" * 80)
print("LOAD BẢNG STUDENT_SCORES")
print("=" * 80)

# Đọc từ nhiều năm
years = [2021,2022,2023,2024,2025]
all_dfs = []
for year in years:
    try:
        df_year = spark.read.option("header", "true").option("inferSchema", "false").option("encoding", "UTF-8").csv(f"s3a://bronze/structured_data/điểm từng thí sinh/{year}/*.csv").withColumn("Year", lit(year))
        all_dfs.append(df_year)
        print(f"Đọc được {df_year.count():,} dòng từ năm {year}")
    except:
        print(f"Không tìm thấy dữ liệu năm {year}")

df_scores = all_dfs[0]
for df in all_dfs[1:]:
    df_scores = df_scores.union(df)

# Đọc bảng subject để map tên môn -> subjectId
df_subject_lookup = spark.table("nessie.silver_tables.subject").select("subjectId", "subjectName")
subject_map = {row.subjectName: row.subjectId for row in df_subject_lookup.collect()}
print(f"\nĐã load {len(subject_map)} môn học để mapping")

# UDF để parse điểm và map với subjectId
from typing import Dict
def parse_scores_with_subject_id(score_string: str) -> Dict[int, float]:
    if not score_string or score_string.strip() == "":
        return {}
    scores_dict = {}
    try:
        pairs = score_string.split(",")
        for pair in pairs:
            if ":" in pair:
                subject_name, score = pair.split(":")
                subject_name = subject_name.strip()
                # Map tên môn -> subjectId
                if subject_name in subject_map:
                    subject_id = subject_map[subject_name]
                    try:
                        scores_dict[subject_id] = float(score.strip())
                    except:
                        pass
    except:
        pass
    return scores_dict

parse_scores_udf = udf(parse_scores_with_subject_id, MapType(IntegerType(), DoubleType()))

# Transform
df_student_scores_silver = df_scores.withColumn("studentId", concat(col("SBD"), col("Year").cast("string"))).withColumn("scores", parse_scores_udf(col("DiemThi"))).withColumn("regionId", substring(col("SBD"), 1, 2).cast("string")).select(
    col("studentId").cast("string"),
    col("regionId").cast("string"),
    col("Year").cast("int").alias("year"),
    col("scores"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("studentId").isNotNull() & col("year").isNotNull() & col("scores").isNotNull()).dropDuplicates(["studentId"])

df_student_scores_silver.writeTo("nessie.silver_tables.student_scores").using("iceberg").createOrReplace()
print(f"Đã ghi {df_student_scores_silver.count():,} dòng vào student_scores")

# Verify
print("\nXem mẫu dữ liệu (scores giờ là Map<subjectId, score>):")
spark.table("nessie.silver_tables.student_scores").show(5, truncate=False)
spark.table("nessie.silver_tables.student_scores").groupBy("year").count().orderBy("year").show()

LOAD BẢNG STUDENT_SCORES
Đọc được 993,901 dòng từ năm 2021


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/local/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


Không tìm thấy dữ liệu năm 2022
Đọc được 1,025,333 dòng từ năm 2023
Đọc được 1,061,466 dòng từ năm 2024
Đọc được 1,152,914 dòng từ năm 2025

Đã load 51 môn học để mapping


Đã ghi 5,190,113 dòng vào student_scores

Xem mẫu dữ liệu (scores giờ là Map<subjectId, score>):
+------------+--------+----+--------------------------------------------------------------------+--------------------------+--------------------------+
|studentId   |regionId|year|scores                                                              |created_at                |updated_at                |
+------------+--------+----+--------------------------------------------------------------------+--------------------------+--------------------------+
|010000022022|01      |2022|{33 -> 8.5, 50 -> 7.5, 4 -> 8.25, 41 -> 8.4, 43 -> 6.75, 14 -> 7.6} |2025-11-28 04:35:11.429135|2025-11-28 04:35:11.429135|
|010000072022|01      |2022|{33 -> 8.0, 50 -> 7.5, 4 -> 9.0, 41 -> 7.2, 43 -> 6.0, 14 -> 5.0}   |2025-11-28 04:35:11.429135|2025-11-28 04:35:11.429135|
|010000082025|01      |2025|{33 -> 4.0, 50 -> 4.0, 43 -> 6.25}                                  |2025-11-28 04:35:11.429135|2025-11-28 04:35:11

## 9. Load Bảng ARTICLE và COMMENT từ TikTok Data

In [ ]:
import re
from datetime import datetime
from pyspark.sql.functions import regexp_extract, split, size, when

print("=" * 80)
print("LOAD BẢNG ARTICLE VÀ COMMENT TỪ TIKTOK DATA")
print("=" * 80)

# Đọc tất cả file CSV trong thư mục tiktok-data
tiktok_files = [
    "s3a://bronze/tiktok-data/tiktok_comments_2025-11-07T07-09-34.csv",
    "s3a://bronze/tiktok-data/tiktok_comments_2025-11-07T07-11-15.csv"
]

# Khởi tạo ID counter cho article
article_counter = 1
comment_counter = 1

# Danh sách để chứa tất cả article và comment
all_articles = []
all_comments = []

for file_path in tiktok_files:
    try:
        print(f"\nXử lý file: {file_path}")
        
        # Đọc file CSV với encoding UTF-8
        df_raw = spark.read.option("header", "false").option("encoding", "UTF-8").csv(file_path)
        
        # Lấy thông tin metadata từ các dòng đầu
        rows = df_raw.collect()
        
        # Parse thông tin article từ metadata
        post_url = ""
        author = ""
        tag_name = ""
        author_url = ""
        time_publish = ""
        like_count = 0
        comment_count = 0
        share_count = 0
        title = ""  # Mô tả của bài đăng
        
        for row in rows[:15]:  # Chỉ xem 15 dòng đầu để lấy metadata
            line = row[0] if row[0] else ""
            
            if "Post URL:" in line:
                post_url = line.split("Post URL:")[1].strip()
            elif "Người đăng:" in line:
                author = line.split("Người đăng:")[1].strip()
            elif "Tag người đăng:" in line:
                tag_name = line.split("Tag người đăng:")[1].strip()
            elif "URL người đăng:" in line:
                author_url = line.split("URL người đăng:")[1].strip()
            elif "Thời gian đăng:" in line:
                time_str = line.split("Thời gian đăng:")[1].strip()
                # Convert format "2-7-2024" to timestamp
                try:
                    time_publish = datetime.strptime(time_str, "%d-%m-%Y")
                except:
                    time_publish = datetime.now()
            elif "Số lượt tym:" in line:
                tym_str = line.split("Số lượt tym:")[1].strip()
                try:
                    if "K" in tym_str:
                        like_count = int(float(tym_str.replace("K", "")) * 1000)
                    else:
                        like_count = int(tym_str)
                except:
                    like_count = 0
            elif "Số lượt comment:" in line:
                try:
                    comment_count = int(line.split("Số lượt comment:")[1].strip())
                except:
                    comment_count = 0
            elif "Số lượt share:" in line:
                share_str = line.split("Số lượt share:")[1].strip()
                try:
                    if share_str != "N/A":
                        share_count = int(share_str)
                    else:
                        share_count = 0
                except:
                    share_count = 0
            elif "Mô tả của bài đăng:" in line:
                title = line.split("Mô tả của bài đăng:")[1].strip().strip('"')
        
        # Tạo record cho bảng article
        article_data = {
            "articleID": article_counter,
            "title": title,
            "description": None,  # Để null như yêu cầu
            "author": author,
            "url": post_url,
            "timePublish": time_publish,
            "likeCount": like_count,
            "commentCount": comment_count,
            "shareCount": share_count,
            "type": "TikTok",
            "created_at": datetime.now(),
            "updated_at": datetime.now()
        }
        all_articles.append(article_data)
        
        # Tìm dòng header của comment (STT,Tên,Tag tên,...)
        header_row_index = -1
        for i, row in enumerate(rows):
            if row[0] and "STT,Tên,Tag tên" in row[0]:
                header_row_index = i
                break
        
        if header_row_index >= 0:
            # Đọc lại file từ dòng comment data
            df_comments_raw = spark.read.option("header", "true").option("encoding", "UTF-8").csv(file_path)
            
            # Lọc chỉ lấy các dòng là comment (bắt đầu từ dòng có STT)
            df_comments = df_comments_raw.filter(
                col("STT").cast("int").isNotNull() & 
                col("STT") != "STT"
            )
            
            # Thu thập comment data
            comment_rows = df_comments.collect()
            
            for row in comment_rows:
                try:
                    # Parse thời gian comment
                    comment_time_str = row["Time"] if row["Time"] else ""
                    comment_time = None
                    try:
                        comment_time = datetime.strptime(comment_time_str, "%d-%m-%Y")
                    except:
                        comment_time = datetime.now()
                    
                    # Xác định level comment
                    level_comment = 2 if row["Level Comment"] == "Yes" else 1
                    
                    # Parse reply info
                    reply_to = row["Replied To Tag Name"] if row["Replied To Tag Name"] and row["Replied To Tag Name"] != "---" else None
                    
                    comment_data = {
                        "commentID": comment_counter,
                        "articleID": article_counter,
                        "name": row["Tên"] if row["Tên"] else "",
                        "tagName": row["Tag tên"] if row["Tag tên"] else "",
                        "urlUser": row["URL"] if row["URL"] else "",
                        "comment": row["Comment"] if row["Comment"] else "",
                        "commentTime": comment_time,
                        "commentLike": int(row["Likes"]) if row["Likes"] and row["Likes"].isdigit() else 0,
                        "levelComment": level_comment,
                        "replyTo": reply_to,
                        "numberOfReply": int(row["Number of Replies"]) if row["Number of Replies"] and row["Number of Replies"].isdigit() else 0,
                        "created_at": datetime.now(),
                        "updated_at": datetime.now()
                    }
                    all_comments.append(comment_data)
                    comment_counter += 1
                except Exception as e:
                    print(f"Lỗi xử lý comment: {e}")
                    continue
        
        article_counter += 1
        print(f"Đã xử lý xong file {file_path}")
        
    except Exception as e:
        print(f"Lỗi xử lý file {file_path}: {e}")
        continue

print(f"\nTổng số article: {len(all_articles)}")
print(f"Tổng số comment: {len(all_comments)}")

In [ ]:
# Tạo DataFrame cho article
if all_articles:
    article_schema = StructType([
        StructField("articleID", IntegerType(), False),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("author", StringType(), True),
        StructField("url", StringType(), True),
        StructField("timePublish", TimestampType(), True),
        StructField("likeCount", IntegerType(), True),
        StructField("commentCount", IntegerType(), True),
        StructField("shareCount", IntegerType(), True),
        StructField("type", StringType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])
    
    df_article_silver = spark.createDataFrame(all_articles, article_schema)
    
    # Ghi vào bảng article
    df_article_silver.writeTo("nessie.silver_tables.article").using("iceberg").createOrReplace()
    print(f"Đã ghi {df_article_silver.count()} dòng vào bảng article")
    
    # Verify
    spark.table("nessie.silver_tables.article").show(5, truncate=False)
else:
    print("Không có dữ liệu article để ghi")

In [ ]:
# Tạo DataFrame cho comment
if all_comments:
    comment_schema = StructType([
        StructField("commentID", IntegerType(), False),
        StructField("articleID", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("tagName", StringType(), True),
        StructField("urlUser", StringType(), True),
        StructField("comment", StringType(), True),
        StructField("commentTime", TimestampType(), True),
        StructField("commentLike", IntegerType(), True),
        StructField("levelComment", IntegerType(), True),
        StructField("replyTo", StringType(), True),
        StructField("numberOfReply", IntegerType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])
    
    df_comment_silver = spark.createDataFrame(all_comments, comment_schema)
    
    # Ghi vào bảng comment
    df_comment_silver.writeTo("nessie.silver_tables.comment").using("iceberg").createOrReplace()
    print(f"Đã ghi {df_comment_silver.count()} dòng vào bảng comment")
    
    # Verify
    spark.table("nessie.silver_tables.comment").show(5, truncate=False)
    
    # Thống kê theo article
    print("\nThống kê comment theo article:")
    spark.table("nessie.silver_tables.comment").groupBy("articleID").count().orderBy("articleID").show()
else:
    print("Không có dữ liệu comment để ghi")